In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import xgboost as xg
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from matplotlib import pyplot
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.utils import shuffle

df_train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
df_sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

df_train.drop('id', inplace=True, axis=1)

y = df_train['target'].values
X = df_train.drop('target', axis=1)
print(y)
print(df_sub['id'].values == df_test['id'].values)
df_test.drop('id', inplace=True, axis=1)
print('Train Data:', df_train.shape)
print('Test Data:', df_test.shape)
X_test = df_test
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[7.24304259 8.20333114 7.77609076 ... 9.59248653 8.2079506  8.89028481]
[ True  True  True ...  True  True  True]
Train Data: (300000, 15)
Test Data: (200000, 14)


In [ ]:
estimators = [
{'name': 'RandomForest',
'instance': RandomForestRegressor(n_estimators=300, oob_score=True, n_jobs = -1, random_state=42,
criterion= 'mse', max_features = 'auto', min_samples_leaf = 1)},
{'name': 'GBR',
'instance': GradientBoostingRegressor(loss='ls', alpha=0.95, n_estimators=300)},
{'name': 'ABR',
'instance': AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
n_estimators=300, random_state=np.random.RandomState(1))},
{'name': 'LSR',
'instance': Lasso(alpha=1e-8,normalize=True, max_iter=1e5)},
{'name': 'ETR',
'instance': ExtraTreesRegressor(n_estimators=300)}
]

In [ ]:
def get_best_params(X, y):
    param_grid = { 'n_estimators' : [200, 300, 500],
        'max_depth' : [2, 3, 5],
        'oob_score' : [True, False]
    }
    rf = RandomForestRegressor(n_estimators=200, oob_score=True, n_jobs = -1, random_state=42, bootstrap='True',
    criterion= 'mse', max_features = 'auto', min_samples_leaf = 50)
    CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 10)
    CV_rfc.fit(X, y)
    print (CV_rfc.best_params_)
    return CV_rfc.best_params_

def train_dataOld(X_train, y_train, X_val, y_val, estimators):
    est = {}
    for estimator in estimators:
        estimator['instance'].fit(X_train, y_train)
        cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=10, random_state=42)
        val_errs = np.sqrt(cross_val_score(estimator=estimator['instance'], X=X_val, y=y_val, cv=cv, scoring='neg_mean_squared_error') * -1)
        print(f"validation error: {val_errs.mean()}, std dev: {val_errs.std()}")
        est[estimator[‘instance’]] = rmse
        est[estimator[‘instance’]] = val_errs.mean()
    model = min(iter(est.keys()), key=lambda k: est[k])
    print('Lowest rmse:', est[model])
    print('winner model:', model)
    return model

In [ ]:
# split data into train and test sets
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)
print(X.shape)
print(X_test.shape)
# Model training process:
X = xg.DMatrix(X, label=y)
X_test = xg.DMatrix(X_test)
xgb_params = {
'seed': 0,
'colsample_bytree': 0.8, #changed from 0.8
'silent': 1,
'gamma': 0.3, #changed from 0.1
'reg_alpha': 0.3,
'subsample': 0.8, #changed from 0.8
'learning_rate': 0.01, #changed from 0.01
'objective': 'reg:linear',
'max_depth': 1,
'lambda': 0.5,
'num_parallel_tree': 1,
'min_child_weight': 10,
'eval_metric': 'rmse'
}
res = xg.cv(xgb_params, X, num_boost_round=1000, nfold=10, seed=0, stratified=False,
early_stopping_rounds = 25, verbose_eval=10, show_stdv = True)
best_nrounds = res.shape[0]-1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]
print("Ensemble CV: {0}+{1}".format(cv_mean, cv_std))
gbdt = xg.train(xgb_params, X, best_nrounds)
# Print the r2 score
print(r2_score(y, gbdt.predict(X)))
# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, gbdt.predict(X)))
print("rmse: {}".format(rmse))

In [ ]:
#Parameter search using GridSearchCV for XgBoost using scikit learn XGBoostRegreesor API:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)], 'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
# Initialize XGB and GridSearch
xgb = xg.XGBRegressor(nthread=-1)
grid = GridSearchCV(xgb, params, n_jobs = -1)
grid.fit(X, y)
# Print the RMSE score
print(grid.best_params_)
print(np.sqrt(mean_squared_error(y, grid.best_estimator_.predict(X))))
# Save the file
y_test = grid.best_estimator_.predict(X_test)
frame = pd.DataFrame({'target': y_test}, index=df_sub['id'])
frame.to_csv('submission.csv')

